<a href="https://colab.research.google.com/github/chandrajitpal/Cybersecurity/blob/main/cyberlabs/LAB4_CYBER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Network Anomaly Detection with AI**

In [ ]:
# prompt: print hello world using rot13

import codecs

# Function to encode a string using ROT13
def rot13_encode(text):
  return codecs.encode(text, 'rot_enc_13')

# String to encode
hello_world = "hello world"

# Encode the string
encoded_hello_world = rot13_encode(hello_world)

# Print the encoded string
encoded_hello_world